In [5]:
import numpy as np
import pandas as pd
from csv import DictReader as DR
import re
import math

file_pointer = pd.read_csv('/content/Data_p3.csv', encoding = 'latin-1')
    
data = np.asarray(file_pointer)

np.random.shuffle(data)
    
x_train = []
y_train = []
x_test = []
y_test = []

data_len = len(data)
for i in range(data_len) :
    
    l = len(x_train)
    if l < 0.7 * data_len :
        x_train.append(data[i][1])
        y_train.append(data[i][0])
    else :
        x_test.append(data[i][1])
        y_test.append(data[i][0])


print("Details of dataset")
print(file_pointer['v1'].value_counts(normalize = True))
print()

print('Details of Training Data')
print(len(x_train))
print(pd.DataFrame(y_train).value_counts(normalize = True))
print()


print('Details of Testing Data')
print(len(x_test))
print(pd.DataFrame(y_test).value_counts(normalize = True))
print()

print()
print()


x_train_len = len(x_train)

for i in range(x_train_len) :
        
    x_train[i] = x_train[i].lower()
    x_train[i] = re.sub(r'[^\w\s]', ' ', x_train[i])
    x_train[i] = x_train[i].split(' ')
    
    temp_x = []
    
    for word in x_train[i] :
        
        if len(word):
            temp_x.append(word)
    
    x_train[i] = temp_x


spam_count = 0
ham_count = 0
spam_word_count = 0
ham_word_count = 0
total_words = 0
spam_words = {}
ham_words = {}
all_words = {}


'''
if var == 0 :
        spam_words[word] = 0 
    else :
        spam_words[word] = (np.exp(-1*((n_sum-mean) |*2)/(2*var)))(1/np.sqrt(2*np.pi*var))
    
    # spam_words[word] = gausian(n_sum,mean,var) + 0.1

'''

for i, msg in enumerate(x_train) :
    
    if y_train[i] == 'spam' :
        spam_count = spam_count + 1
        
        for word in msg :

            total_words = total_words + 1
            spam_word_count = spam_word_count + 1
            
            if word in spam_words :
                spam_words[word] = spam_words[word] + 1
            else :
                spam_words[word] = 1

            if word in all_words :
                all_words[word] = all_words[word] +  1
            else :
                all_words[word] = 1
            
    elif y_train[i] == 'ham' :
        ham_count = ham_count + 1
        
        for word in msg :

            total_words = total_words + 1
            ham_word_count = ham_word_count + 1
            
            if word in ham_words :
                ham_words[word] += 1
            else :
                ham_words[word] = 1

            if word in all_words :
                all_words[word] +=  1
            else :
                all_words[word] = 1
            

print("Analysis of the training data done : ")

print("Total count of Unique words =  ",len(all_words))

print("About HAM words")
print("Count = " , len(ham_words))
print("Words are : " , ham_words)
print()

print("About SPAM words")
print("Count = " , len(spam_words))
print("Words are : " , spam_words)

print()
print()


def gausian(word,m,v):
    if(v==0):
        return 0
    return (np.exp(-1*((word-m)*2)/(2*v)))(1/np.sqrt(2*np.pi*v))

alpha = 1

for word in all_words :
    
    n_sum = 0
    
    if word in spam_words:
        n_sum = spam_words[word]
    
    arr = []
    for msg in x_train :
        
        count = 0
        for wd in msg :
            if wd == word :
                count = count + 1
        
        arr.append(count)
    
    mean = np.mean(arr)
    var = np.var(arr)

    if var == 0 :
        spam_words[word] = 0 
    else :
        spam_words[word] = (np.exp(-1*((n_sum-mean)*2)/(2*var)))*(1/np.sqrt(2*np.pi*var))
    
    # spam_words[word] = gausian(n_sum,mean,var) + 0.1

    n_sum = 0
    
    if word in ham_words :
        n_sum = ham_words[word]
    
    arr = []
    
    for msg in x_train :
        
        count = 0
        
        for wd in msg :
            if wd == word:
                count = count + 1
        
        arr.append(count)
    
    mean = np.mean(arr)
    var = np.var(arr)

    if var == 0 :
        ham_words[word] = 0 
    else :
        ham_words[word] = (np.exp(-1*((n_sum-mean)*2)/(2*var)))*(1/np.sqrt(2*np.pi*var))
    
    # ham_words[word] = gausian(n_sum,mean,var) + 0.1   




def get_prediction_spam(msg) :
    
    spam_prob = spam_count/x_train_len
    ham_prob = ham_count/x_train_len

    copy_msg = msg

    msg = msg.lower()
    msg = re.sub(r'[^\w\s]', ' ', msg)
    msg = msg.split(' ')
    
    temp = []
    for word in msg:
        
        if len(word):
            temp.append(word)
    
    msg = temp  

    words_in_msg = msg
    
    for word in words_in_msg :

        a = (1 / (total_words + len(all_words)) )

        if word in ham_words :
            ham_prob = ham_prob * (ham_words[word] / ham_word_count) / (all_words[word] / total_words)
        else :
            ham_prob = ham_prob * (1 / (ham_word_count + len(ham_words)))
            
            if word in all_words :
                ham_prob = ham_prob / (all_words[word] / total_words)
            else :
                ham_prob = ham_prob / a
        
        if  word in spam_words :
            spam_prob = spam_prob * (spam_words[word] / spam_word_count) / (all_words[word] / total_words)
        else :
            spam_prob = spam_prob * (1 / (spam_word_count + len(spam_words)))
            
            if word in all_words :
                spam_prob = spam_prob / (all_words[word] / total_words)
            else :
                spam_prob = spam_prob / a
    
    if ham_prob <= spam_prob :
        return 1
    
    return 0


Y_predicted = []

for msg in x_test :
    
    if get_prediction_spam(msg) == 0:
        Y_predicted.append('ham')
    else :
        Y_predicted.append('spam')

error_count = 0
l1 = len(y_test)

for i in range(l1):
    
    if Y_predicted[i] != y_test[i] :
        error_count = error_count + 1


error_per = error_count / l1
error_per = error_per * 100
accuracy_per = 100 - error_per
print("Final Results of analysis : ")

print('Error % = ' ,error_per)
print('Accuracy % = ',accuracy_per)

print()
print()

Details of dataset
ham     0.865937
spam    0.134063
Name: v1, dtype: float64

Details of Training Data
3901
ham     0.865163
spam    0.134837
dtype: float64

Details of Testing Data
1671
ham     0.867744
spam    0.132256
dtype: float64



Analysis of the training data done : 
Total count of Unique words =   7198
About HAM words
Count =  5666
Words are :  {'im': 54, 'just': 211, 'wondering': 8, 'what': 206, 'your': 276, 'doing': 70, 'right': 64, 'now': 212, 's': 339, 'only': 95, 'i': 2071, 'took': 14, 'tablets': 3, 'reaction': 2, 'morning': 56, 'hiya': 5, 'stu': 1, 'wot': 17, 'u': 697, 'up': 214, '2': 232, 'in': 602, 'so': 308, 'much': 78, 'truble': 1, 'at': 280, 'home': 119, 'moment': 8, 'evone': 1, 'hates': 1, 'me': 548, 'even': 36, 'the': 772, 'hell': 6, 'av': 2, 'done': 35, 'y': 34, 'wont': 23, 'tell': 72, 'text': 46, 'bck': 2, 'please': 47, 'luv': 23, 'dan': 2, 'cant': 47, 'pick': 51, 'phone': 61, 'pls': 77, 'send': 80, 'a': 735, 'message': 45, 'oh': 73, 'icic': 1, 'k': 98, 'lor':